In [56]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf")




In [83]:
def create_ix(docs):
    ix=dict()
    for id,content in docs.items():

        doc = nlp(content)

        for token in doc:
            if not token.is_stop and token.tag_ !="." :
                lemma = token.lemma_.lower()
                if lemma in ix:
                    if id in ix[lemma]:
                        ix[lemma][id]=(ix[lemma][id][0]+1,len(doc))
                    else:
                        ix[lemma][id]=(1,len(doc))

                else:
                    ix[lemma]={id:(1,len(doc))}
    for word in ix:
        for document in ix[word]:
            #tf * idf
            ix[word][document]= len(ix[word])/len(docs) * ix[word][document][0]/ix[word][document][1]
    return ix

In [84]:
#search
def search_query(query,ix): 
    doc = nlp(query)
    results={}
    for token in doc:
        lemma = token.lemma_.lower()
        if  lemma in ix:
            for id in ix[lemma]:
                if id in results:
                    results[id]+=ix[lemma][id]
                else:
                    results[id]=ix[lemma][id]
    if len(results) > 0:
        return max(results, key=results.get)
    else:
         return None

In [88]:
def match(docs,searches):
    ix=create_ix(docs)
    matching={}
    for id,query in searches.items():
        matching[id]=search_query(query,ix)
    return matching


In [89]:
docs={
"url1":"Apple is looking at buying U.K. startup for $1 billion with a bottle",
"url2":"What is in the bottle? The bottle is in my House. Fuck the startup.",
"url3":"Apple is not a startup."
}

searches={
    "query1":"apple",
    "query2":"bottle",
    "query3":"edek"
}

match(docs,searches)

{'query1': 'url3', 'query2': 'url2', 'query3': None}